In [49]:
import pandas as pd
import numpy as np
import re
import MeCab
data = pd.read_excel('idiom_mod_data.xlsx')

In [51]:
# load raw dataset and drop nan
df = data[['어휘','수정']]
df = df.dropna(axis=0) # 애매한 의미 제거 
df.head(30)

,어휘,수정
0,가꾸로 박히다,머리를 땅으로 향한 채 떨어지다.\n
1,가난(이) 들다,가난하게 되다.
2,가난이 파고들다,더욱더 가난해지다.\n
3,가늠(을) 보다,형편이나 시세 따위를 살피다.
4,가늠(을) 잡다,형편이나 시세 따위를 살피다.
5,가늠이 가다,짐작이나 헤아림이 이루어지거나 미치다.\n
6,가닥을 잡다,"분위기, 상황, 생각 따위를 이치나 논리에 따라 바로잡다.\n"
7,가닥이 잡히다,"분위기, 상황, 생각 따위를 이치나 논리에 따라 바로잡게 하다.\n"
8,가도 오도 못하다,한곳에서 자리를 옮기거나 움직일 수 없는 상태가 되다.\n
9,가락(을) 떼다,흥이 나는 일에 첫 동작을 실행하다.


In [52]:
dics = df.values.tolist()

# Preprocessing

In [53]:
# ()처리 포함, 제거 두 예제로 나눔 
idiom_list_pre = []
meaning_list_pre = []
for d in dics:
    idiom = d[0]
    meaning = d[1]
    parentheses = re.findall('\(.*?\)', idiom) # extract 조사 in ()
    
    if len(parentheses) != 0:
        # 1. () 제외
        idiom_list_pre.append(\
                    re.sub('\(.*?\)', '', idiom))
        meaning_list_pre.append(meaning)
#         print(\
#                 re.sub('\(.*?\)', '', idiom))
        # 2. ()안 조사 포함
        idiom_list_pre.append(\
                re.sub('[\(,\)]', '', idiom))
        meaning_list_pre.append(meaning)
#         print(\
#                 re.sub('[\(,\)]', '', idiom))
    else:
        idiom_list_pre.append(idiom)
        meaning_list_pre.append(meaning)
print(len(idiom_list_pre))
print(len(meaning_list_pre))

5371
5371


In [55]:
# 다중 관용구 처리
idiom_list = []
meaning_list = []
for i, m in zip(idiom_list_pre, meaning_list_pre):
    # 의미 빈거 제거 (애매한 의미 등)
    idiom = i.replace('\n','')
    meaning = m.replace('\n','')
    #print(idiom)
    items = re.findall('\[.*?\]', idiom) #extracts string in bracket[] print items
    parentheses = re.findall('\(.*?\)', idiom) # extract 조사 in ()
    
    if len(items) != 0:
        #print('\n\n')
        #print(idiom)
        #print(items)    
        
        # original
        idiom_list.append(\
                re.sub('\[.*?\]', '', idiom))
        meaning_list.append(meaning)
#         print(\
#                 re.sub('\[.*?\]', '', idiom))
        
        # replace pair set
        rep_list = []
        str_list = idiom.split(' ')
        for word in str_list:
            for rep in items:
                if rep in word:
                    rep_list.append([word.replace(rep,''), rep])
        #print(rep_list)

        for pair in rep_list:
            # 유사어 여러개 있을때
            if '/' in pair[1]:
                reps = re.sub('[\[\]]','',pair[1]).split('/')
                #print(reps)
                for r in reps:
                    idiom_list.append(\
                            re.sub('\[.*?\]', '', (idiom.replace(pair[0]+pair[1], r))))
#                     print(\
#                             re.sub('\[.*?\]', '', (idiom.replace(pair[0]+pair[1], r))))
                    meaning_list.append(meaning)
            else:
                idiom_list.append(\
                        re.sub('\[.*?\]', '',\
                            (idiom.replace(pair[0]+pair[1], re.sub('[\[\]]','', pair[1])))))
#                 print(\
#                         re.sub('\[.*?\]', '',\
#                             (idiom.replace(pair[0]+pair[1], re.sub('[\[\]]','', pair[1])))))
                meaning_list.append(meaning)
    
    else:
        idiom_list.append(idiom)
        meaning_list.append(meaning)
        
print(len(idiom_list))
print(len(meaning_list))

5804
5804


In [56]:
print(idiom_list[0:4])
print(meaning_list[0:4])

['가꾸로 박히다', '가난 들다', '가난이 들다', '가난이 파고들다']
['머리를 땅으로 향한 채 떨어지다.', '가난하게 되다.', '가난하게 되다.', '더욱더 가난해지다.']


# tokenizing with spacing token

In [57]:
# 형태소 분석, "/n, . 처리"
# save idiom dictionary to csv file
spacing_tkn = '__'

In [90]:
# 형태소 분석
# " 허리 가 __꼿꼿 하 ㄴ __철수"
mecab = MeCab.Tagger()

def tokenize(str, spacing_tkn='__'):
    # replace white space to spacing token
    str_spacing = str.replace(' ', spacing_tkn)
    # tokenize
    pos = mecab.parse(str_spacing).split('\n')
    tokens = []
    for p in pos:
        #print(p)
        token = p.split('\t')[0]
        # 연결 어미, 종결 어미 제외
        if "EC" in p or "EF" in p or token == "EOS":
            break
        # 복합 일때
        features = p.split(',')
        if features[-1] != '*':
            for f in features[-1].split('+'):
                tokens.append(f.split('/')[0])
        else:
            tokens.append(token)
    return tokens


In [91]:
tokenized = tokenize("머리를 땅으로 향한 채 떨어지다.")
print(tokenized)
print(' '.join(tokenized))

['머리', '를', '__', '땅', '으로', '__', '향하', 'ᆫ', '__', '채', '__', '떨어지']
머리 를 __ 땅 으로 __ 향하 ᆫ __ 채 __ 떨어지


In [92]:
# tokenizing (idiom, meaning)
idiom_df = pd.DataFrame(columns=['idiom', 'meaning'])
for i, m in zip(idiom_list, meaning_list):
    idiom = ' '.join(tokenize(i))
    meaning = ' '.join(tokenize(m))
    idiom_df = idiom_df.append(pd.DataFrame([[idiom, meaning]], columns=['idiom', 'meaning']), ignore_index=True)

print(len(idiom_df))
idiom_df.head(10)

5804


,idiom,meaning
0,가꾸로 __ 박히,머리 를 __ 땅 으로 __ 향하 ᆫ __ 채 __ 떨어지
1,가난 __ 듣,가난 하
2,가난 이 __ 듣,가난 하
3,가난 이 __ 파고듣,더욱더 __ 가난
4,가늠 __ 보다,형편 이나 __ 시세 __ 따위 를 __ 살피
5,가늠 을 __ 보다,형편 이나 __ 시세 __ 따위 를 __ 살피
6,가늠 __ 잡,형편 이나 __ 시세 __ 따위 를 __ 살피
7,가늠 을 __ 잡,형편 이나 __ 시세 __ 따위 를 __ 살피
8,가늠 이 __ 갈,짐작 이나 __ 헤아리 ᄆ 이 __ 이루어지
9,가닥 을 __ 잡,"분위기 , __ 상황 , __ 생각 __ 따위 를 __ 이치 나 __ 논리 에 __"


In [93]:
# save tokenized idiom, meaning to csv file
idiom_df.to_csv('tokenized_idiom.csv', index=True)